# Results Prototype Notebook

## Framing

This notebook generates the linear probing and inter-class distance metrics for a trained Perceiver Autoencoder model (with or without predictive loss). 

## Design
 - Load the `validate` set from Kinetics 400. 
     - Parse the text file in `Multimodal_Foundation/datasets/Kinetics/frame_cnts_val.txt` to get the actual number of frames. 
     - Load some random segment from that number of frames (use random seed or store this somehow). 
     - Figure out how to make a `tf.Dataset` with labels from this. 
 - Load a trained model, parse the data, generate latent representations of each video in the validation set. 
 - Do a cheeky PCA on the data, take a look. 
 - Figure out how the ViT-MAE paper performed linear probing. 
 - Perform linear probing, compute top-5 accuracy. 

In [1]:
## Import box
import os 
import sys 
import random
import pathlib
import itertools
import collections
import math
import argparse
import datetime
import pickle
import pdb
import time
from packaging import version

os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

sys.path.append("../src/utils") # Adding utils folder to path

import tensorflow as tf 
from tensorflow import keras
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import imageio

import model
import video_loader as vl
import video_preprocess as vp
import train_m2
import parallel_video_loader as pvl

2022-12-21 09:36:03.755520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 09:36:03.898580: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-21 09:36:03.925804: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-21 09:36:04.525545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

ModuleNotFoundError: No module named 'tensorflow_addons'